In [1]:
import json
import base64
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

## Import the data

In [2]:
json_file_path = 'complete_pull.json'
with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

#### Check what it looks like

In [3]:
contents['1']['data'][7]

{'author_id': '1210121128867454977',
 'id': '1591877516427681793',
 'lang': 'en',
 'text': 'Started watching adventure time, there’s no way this show was for children??',
 'edit_history_tweet_ids': ['1591877516427681793']}

## Supporting functionality 

In [ ]:
def chrome_takeFullScreenshot(driver) :
    """ From https://stackoverflow.com/questions/45199076 """
    
    def send(cmd, params):
        resource = "/session/%s/chromium/send_command_and_get_result" % driver.session_id
        url = driver.command_executor._url + resource
        body = json.dumps({'cmd':cmd, 'params': params})
        response = driver.command_executor._request('POST', url, body)
        return response.get('value')

    def evaluate(script):
        response = send('Runtime.evaluate', {'returnByValue': True, 'expression': script})
        return response['result']['value']

    metrics = evaluate( \
    "({" + \
      "width: Math.max(window.innerWidth, document.body.scrollWidth, document.documentElement.scrollWidth)|0," + \
      "height: Math.max(innerHeight, document.body.scrollHeight, document.documentElement.scrollHeight)|0," + \
      "deviceScaleFactor: window.devicePixelRatio || 1," + \
      "mobile: typeof window.orientation !== 'undefined'" + "})") 
    send('Emulation.setDeviceMetricsOverride', metrics)
    screenshot = send('Page.captureScreenshot', {'format': 'png', 'fromSurface': True})
    send('Emulation.clearDeviceMetricsOverride', {})

    return base64.b64decode(screenshot['data'])

In [ ]:
def get_screenshot_and_save(fp):
    png = chrome_takeFullScreenshot(driver)

    with open(fp, 'wb') as f:
        f.write(png)
        f.close()

## Set up the webdriver

In [ ]:
user_data_dir = r"--user-data-dir=/home/username/.config/google-chrome/Default" #change username as required
profile_directory = r'--profile-directory=Default'

options = webdriver.ChromeOptions()
options.add_argument(user_data_dir)
options.add_argument(profile_directory)
driver = webdriver.Chrome(chrome_options=options)

## Run the scrape

In [ ]:
page_start = "https://twitter.com/"
for keys in contents:
    for tweet in contents[keys]['data']:
        author = str(tweet['author_id'])
        tweet_number = str(tweet['id'])
        url = page_start+author+"/status/"+tweet_number
        driver.get(url)
        time.sleep(5) #allows image heavy pages to load
        get_screenshot_and_save("./output/author_"+author+"_tweet_"+tweet_number+".png")